
```
---
title: API Review with Foursquare API
type:  lesson + lab + demo
duration: "1:25"
creator:
    name: David Yerrington
    city: SF
---
```
<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 10px">

#  API Demo / Lab + NLP
Week 8 | 1.3


<img src="https://snag.gy/RNAEgP.jpg" width="600">

Can we correctly identify which of these two old men tweeted what?


## (5 mins) Opening 

Today we are going to attempt to classify wether a tweet comes from Trump, or Sanders.  We are going to:

- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

![](https://snag.gy/HPBQbJ.jpg)

We now will now go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/), just like we did for Foursquare.  After we set up our app, we will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

Someone was nice enough to build a nice libary for us in Python that we only need to plug in our keys and start collecting data with.  The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [1]:
!pip install twitter python-twitter --upgrade

Requirement already up-to-date: twitter in /usr/local/lib/python2.7/site-packages
Requirement already up-to-date: python-twitter in /usr/local/lib/python2.7/site-packages
Requirement already up-to-date: requests in /usr/local/lib/python2.7/site-packages (from python-twitter)
Requirement already up-to-date: future in /usr/local/lib/python2.7/site-packages (from python-twitter)
Requirement already up-to-date: requests-oauthlib in /usr/local/lib/python2.7/site-packages (from python-twitter)
Requirement already up-to-date: oauthlib>=0.6.2 in /usr/local/lib/python2.7/site-packages (from requests-oauthlib->python-twitter)


## Some Boring Twitter Rules

Twitter says they will rate limit your requests:

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rulez":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys

Take note of our application keys that we will be using with our little application that will be connecting to Twitter and mining Tweets from the official Bernie Sanders and Donald Trump twitter accounts.

![](https://snag.gy/H1djQK.jpg)

In [2]:
# import yaml
# credentials = yaml.load(open('credentials.yml'))

# twitter_key = credentials['twitter_key']
# twitter_secret = credentials['twitter_secret']
# token = credentials['twitter_token']
# token_secret = credentials['twitter_tokensecret']

# import twitter
# auth = twitter.OAuth(token, token_secret, twitter_key, twitter_secret)
# twitter_api = twitter.Twitter(auth=auth)

In [3]:
# def stwitter(search_string= '#ASC2016',count=10):
#     search_results = twitter_api.search.tweets(
#                                         q=search_string
#                                         , count=count)
#     data = []
#     for item in search_results['statuses']:
#         mined = {
#             'tweet_id': item.id,
#             'handle': item.user.name,
#             'retweet_count': item.retweet_count,
#             'text': item.text,
#             'mined_at': datetime.datetime.now(),
#             'created_at': item.created_at,
#         }

#         data.append(mined)
#     return data

## Tweet Miner Class Setup

The following code is meant to get us up and running with connectivity to twitter, and the ability to make requests and easily transform the JSON responses to DataFrames.  We will be using object oriented Python in order to organize our code.  We may go into review since this was a topic we covered earlier in the class but we can review it during the lab for those who want to know more about it.


> "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding any rate limit blocks.

#### Key Setup

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the "generate tokens" button to get this
- **access_token_secret** - Also available after "generate tokens" is pressed


In [4]:
# twitter_keys = {
#     'consumer_key':        'NX0QYOJxZtCvA0AtZT6D8o4ku',
#     'consumer_secret':     '4XwbCowqcmL5WxuWwDPMaqnSrcKmAxd17j5jgnekOAwPYqDpeR',
#     'access_token_key':    '212421653-VRsbECJmLx5HqKriWMGrMBT7ZGgFlPz9wAzpnRoU',
#     'access_token_secret': 'pmOcl0KZwy88ZT67eJb5ed5yzYpVxgZWxo1qcs9s4g0ji'
# }


# thisis = twitter.api(
#             consumer_key         =   twitter_keys['consumer_key'],
#             consumer_secret      =   twitter_keys['consumer_secret'],
#             access_token_key     =   twitter_keys['access_token_key'],
#             access_token_secret  =   twitter_keys['access_token_secret']
#         )

# twitter.api



In [73]:
import twitter, re, datetime, pandas as pd

class twitterminer():

    request_limit   =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'NX0QYOJxZtCvA0AtZT6D8o4ku',
        'consumer_secret':     '4XwbCowqcmL5WxuWwDPMaqnSrcKmAxd17j5jgnekOAwPYqDpeR',
        'access_token_key':    '212421653-VRsbECJmLx5HqKriWMGrMBT7ZGgFlPz9wAzpnRoU',
        'access_token_secret': 'pmOcl0KZwy88ZT67eJb5ed5yzYpVxgZWxo1qcs9s4g0ji'
    }
    
    def __init__(self,  request_limit = 20):
        
        self.request_limit = request_limit
        
        # This sets the twitter API object for use internall within the class
        self.set_api()
        
    def set_api(self):
        
        self.api = twitter.Api(
            consumer_key         =   self.twitter_keys['consumer_key'],
            consumer_secret      =   self.twitter_keys['consumer_secret'],
            access_token_key     =   self.twitter_keys['access_token_key'],
            access_token_secret  =   self.twitter_keys['access_token_secret']
        )

    def mine_user_tweets(self, user="dyerrington", max_id=98924, mine_rewteets=False):

        statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.request_limit, max_id=max_id)
        data       =   []
        
        for item in statuses:

            mined = {
                'tweet_id': item.id,
                'handle': item.user.name,
                'retweet_count': item.retweet_count,
                'text': item.text,
                'mined_at': datetime.datetime.now(),
                'created_at': item.created_at,
            }
            
            data.append(mined)
            
        return data
    
    def mine_user_tweets_CL(self, user="dyerrington", mine_rewteets=False):
        data       =   []
        
        for i in range (1,15):
            if i == 1:
                statuses   =   self.api.GetUserTimeline(screen_name=user, 
                                                count=self.request_limit)
                for item in statuses:
                    mined = {
                        'tweet_id': item.id,
                        'handle': item.user.name,
                        'retweet_count': item.retweet_count,
                        'text': item.text,
                        'mined_at': datetime.datetime.now(),
                        'created_at': item.created_at,
                    }
                    data.append(mined)
            
            else: 
                statuses   =   self.api.GetUserTimeline(screen_name=user, 
                                                count=self.request_limit,
                                                max_id=data[-1]['tweet_id'])
                for item in statuses:
                    mined = {
                        'tweet_id': item.id,
                        'handle': item.user.name,
                        'retweet_count': item.retweet_count,
                        'text': item.text,
                        'mined_at': datetime.datetime.now(),
                        'created_at': item.created_at,
                    }
                    data.append(mined)
        
        return data

## Does anyone remember how we "instantiate" a new instance of this class?

**Bonus bonus** How do we call the method to *mine_user_tweets()*?

In [74]:
# twitter ids:  realDonaldTrump, berniesanders
# Let's test this out here..
tm = twitterminer()
tm.mine_user_tweets()

[]

##  Now we create some training data

We will have to munge a little bit in order to get our "mined" data from the Twitter API.  

 - Mine Trump Tweets
 - Create DataFrame
 - Mine Sanders Tweets
 - Append to DataFrame

In [75]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = twitterminer(request_limit=400)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [42]:
trump_df = pd.DataFrame(trump_tweets)
trump_df.head(10)

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Mon Sep 19 19:52:00 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789132,3644,Philly FOP Chief On Presidential Endorsement: ...,777958440211771392
1,Mon Sep 19 16:53:42 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789144,6501,Hillary Clinton's weakness while she was Secre...,777913567676866560
2,Mon Sep 19 16:41:15 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789148,10633,Once again someone we were told is ok turns ou...,777910435425226753
3,Mon Sep 19 16:32:32 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789151,10979,Great job once again by law enforcement! We ar...,777908242538196992
4,Mon Sep 19 12:27:28 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789154,6249,"""@TarukMatuk: @CNN @FoxNews @realDonaldTrump @...",777846568741441536
5,Mon Sep 19 12:14:59 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789158,3760,"""@AngPiazza: @foxandfriends @realDonaldTrump ...",777843428449251328
6,Mon Sep 19 11:02:30 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789161,1796,Will be on @foxandfriends at 7:02 A.M. Enjoy.,777825186758418432
7,Mon Sep 19 02:32:03 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789164,11574,"Terrible attacks in NY, NJ and MN this weekend...",777696726933180416
8,Mon Sep 19 02:30:34 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789167,16461,"Under the leadership of Obama &amp; Clinton, A...",777696356211326976
9,Sun Sep 18 21:11:48 +0000 2016,Donald J. Trump,2016-09-19 15:23:27.789170,11006,HAPPY BIRTHDAY - to the United States Air Forc...,777616135856545792


## Any interesting ngrams going on with Trump?

In [150]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,2))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'https co', 1290),
 (u'thank you', 248),
 (u'will be', 214),
 (u'trump2016 https', 164),
 (u'crooked hillary', 156),
 (u'of the', 150),
 (u'hillary clinton', 129),
 (u'great again', 103),
 (u'make america', 94),
 (u'in the', 93),
 (u'america great', 90),
 (u'makeamericagreatagain https', 79),
 (u'ted cruz', 76),
 (u'makeamericagreatagain trump2016', 74),
 (u'for the', 73),
 (u'to the', 71),
 (u'to be', 68),
 (u'we will', 64),
 (u'the people', 54),
 (u'be interviewed', 54)]

## (10 mins) Try this exercize with Bernie Sanders..

In [44]:
sanders_tweets = miner.mine_user_tweets("berniesanders")

In [45]:
all_tweets = pd.DataFrame(trump_tweets + sanders_tweets)

## Preprocessing our Tweets

In order to do classfication recall that we need a set of features.  Our features are literally what our presidential hopefulls say on Twitter. 

We will need to:
- Vectorize input text data
- Intialize a model (let's try Logistic regression)
- Train / Predict / Cross Validate
- Score / Evaluate


In [77]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(lowercase=True, strip_accents='unicode')
X_all = tfv.fit_transform(all_tweets['text'])

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X_all, all_tweets['handle'])



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Check Prediction vs Random Sanders Tweet

In [78]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

X_all = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(X_all)

array([[ 0.73956231,  0.26043769],
       [ 0.3318663 ,  0.6681337 ]])

## Lab Time

We would like you to perform an analysis using a proper cross validation.  Also, try classfication using other models.

### 1. Implement the same analysis using more data.

Experiment with using more data.  The API may not like that you are blowing through their limits so definitely be careful.  Try to grab only what you need 1x, then work on the copy of the objects that are returned.  Read the documents about rate limits and see if you can get enough without hitting the rate limit.  Are there any options availabl in the API to avoid such a problem?

In [81]:
miner = twitterminer(request_limit=5000)
trump_tweets = miner.mine_user_tweets_CL("realDonaldTrump")
bernie_tweets = miner.mine_user_tweets_CL("berniesanders")

In [84]:
print len(trump_tweets)
trump_df = pd.DataFrame(trump_tweets)
bernie_df = pd.DataFrame(bernie_tweets)
trump_df.head(10)

print len(trump_df.tweet_id.unique())
print trump_df.tweet_id.min()
print trump_df.tweet_id.max()

# trump_tweets2 = miner.mine_user_tweets_CL("realDonaldTrump")
# trump_df2 = pd.DataFrame(trump_tweets2)
# print trump_df2.tweet_id.min()
# print trump_df2.tweet_id.max()


2799
2786
692054432905850884
777999965847220224


In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,3), stop_words='english')

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'trump2016 https', 164),
 (u'crooked hillary', 156),
 (u'hillary clinton', 129),
 (u'make america', 94),
 (u'america great', 90),
 (u'makeamericagreatagain https', 79),
 (u'ted cruz', 76),
 (u'makeamericagreatagain trump2016', 74),
 (u'make america great', 74),
 (u'makeamericagreatagain trump2016 https', 68),
 (u'crooked hillary clinton', 61),
 (u'donald trump', 42),
 (u'south carolina', 39),
 (u'new hampshire', 39),
 (u'new york', 39),
 (u'americafirst https', 36),
 (u'imwithyou https', 35),
 (u'marco rubio', 34),
 (u'lyin ted', 34),
 (u'bernie sanders', 28)]

In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,3), stop_words='english')

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'political revolution', 89),
 (u'wall street', 81),
 (u'health care', 71),
 (u'bernie sanders', 70),
 (u'united states', 56),
 (u'https rt', 46),
 (u'new york', 44),
 (u'climate change', 42),
 (u'vote bernie', 38),
 (u'donald trump', 36),
 (u'puerto rico', 34),
 (u'social security', 32),
 (u'watch live', 31),
 (u'american people', 30),
 (u'feelthebern https', 29),
 (u'minimum wage', 29),
 (u'hillary clinton', 28),
 (u'middle class', 27),
 (u'tax breaks', 27),
 (u'billionaire class', 27)]

In [133]:
all_tweets = pd.DataFrame(bernie_tweets + trump_tweets)

from sklearn.linear_model import LogisticRegression
import numpy as np
y_all = np.array([1 if x == 'Bernie Sanders' else 0 for x in all_tweets['handle'] ])
print X_all.shape, y_all.shape

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(lowercase=True, strip_accents='unicode', stop_words='english', ngram_range=(1,1))
X_all = tfv.fit_transform(all_tweets['text'])




(5597, 84377) (5597,)


### 2. Implement K-Folds or test/train split.

Double check that you are getting random data before moving forward.  What would happen if you over sample Trump more than Sanders?

In [153]:
from sklearn.cross_validation import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3)

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X_train, y_train)

score = cross_val_score(estimator, X_train, y_train, cv=5)
print score, np.mean(score)

[ 0.91454082  0.93622449  0.93997446  0.91954023  0.9412516 ] 0.930306317929


### 3. Mine more Tweets that aren't in your data set
Or use the hold-out method to do a proper test.  Refer back to our advanced classification evaluation lesson if you need to.

In [144]:
y_hat_test = estimator.predict(X_test)
print X_test.shape
print y_hat_test.shape
print y_test.shape

(1680, 10957)
(1680,)
(1680,)


In [145]:
print y_hat_test

[1 0 1 ..., 0 0 1]


### 4. Check your classification report
How's precision / recall of your model?

In [151]:
from sklearn.metrics import classification_report
print 
cls_rep = classification_report(np.ravel(y_test), np.ravel(y_hat_test))

print cls_rep



             precision    recall  f1-score   support

          0       0.94      0.92      0.93       844
          1       0.92      0.94      0.93       836

avg / total       0.93      0.93      0.93      1680



### 5.  Change out your TFIDF vectorizer for CountVectorizer.
How has this impacted your mode performance at all?

In [154]:
all_tweets = pd.DataFrame(bernie_tweets + trump_tweets)

from sklearn.linear_model import LogisticRegression
import numpy as np
y_all = np.array([1 if x == 'Bernie Sanders' else 0 for x in all_tweets['handle'] ])
print X_all.shape, y_all.shape

# Preprocess our text data to Tfidf
cv = CountVectorizer(lowercase=True, strip_accents='unicode', stop_words='english', ngram_range=(1,1))
X_all = cv.fit_transform(all_tweets['text'])

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3)

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X_train, y_train)

score = cross_val_score(estimator, X_train, y_train, cv=5)
print score, np.mean(score)


(5597, 10957) (5597,)
[ 0.94394904  0.93997446  0.91315453  0.92975734  0.91954023] 0.929275121816


### 6.  Implement a different classification method such as random forrests.
Or pick one of your favorites

In [155]:
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier()
estimator.fit(X_train, y_train)

score = cross_val_score(estimator, X_train, y_train, cv=5)
print score, np.mean(score)


[ 0.9044586   0.90038314  0.90166028  0.899106    0.899106  ] 0.900942805314


### 7.  Try to remove stopwords from your text during your preprocessing step

Then double check your classfication report.  Have things improved?

### 8.  Try removing samples that have links or that are obviously just announcements or "noise" that doesn't appear to represent "True" tweets by the authors.

### 9. What are some contrasting words or phrases that you can see between the ngrams for each author?

### 10.  What do you think you can do to improve the scores further?

### 11. **BONUS** Using TextBlob, add a sentiment feature to your dataset.

### 12. BONUS BONUS Apply PCA to your text features
Is this effective? (ie: we could talk about LDA here a little bit)

## Closing

- What where the most impactful changes that helped your models?
- What do you think would happen if we had more Trump Tweets than Sanders?
- What other projects might you think to apply these problems against?